In [34]:
import pandas as pd
import seaborn as sns; sns.set('notebook')
import matplotlib.pyplot as plt
params = {'legend.fontsize': 'large',
          'figure.figsize': (16, 9),
         'axes.labelsize': 'large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'medium',
         'ytick.labelsize':'medium'}
plt.rcParams.update(params)

In [35]:
df = pd.read_csv('densenet_inference_test.csv', index_col=0)
df.mean()

exit           0.973000
prediction    48.845000
target        49.500000
correct        0.823600
score          0.009323
time           0.041526
dtype: float64

In [36]:
data = df.to_numpy()

In [37]:
df = pd.DataFrame(data[1:], columns=['exit', 'prediction', 'target', 'correct', 'score', 'time'])

In [38]:
df.mean()

exit           0.973195
prediction    48.854771
target        49.509902
correct        0.823565
score          0.009166
time           0.032043
dtype: float64

In [39]:
len(df.loc[df['exit']==0])

1725

In [40]:
len(df.loc[(df['exit']==0) & (df['correct'] == False)])

219

In [41]:
len(df.loc[df['exit']==1])

2072

In [42]:
len(df.loc[df['exit']==2])

813

In [49]:
len(df.loc[df['exit']==3])

389

In [44]:
len(df.loc[(df['exit']==3) & (df['correct'] == False)])

177

In [45]:
len(df.loc[(df['correct'] == False)])

882

In [48]:
df

,exit,prediction,target,correct,score,time
0,1,0,0,True,0.917673,0.0333769
1,0,0,0,True,0.971824,0.0186649
2,0,0,0,True,0.922388,0.0164664
3,0,0,0,True,0.917989,0.0179863
4,0,0,0,True,0.844469,0.0190142
5,0,0,0,True,0.999041,0.018966
6,0,0,0,True,0.999265,0.0197375
7,1,0,0,True,0.964632,0.0321687
8,2,0,0,True,0.76264,0.0552192
9,0,0,0,True,0.997576,0.0186883
